# 환경 설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install konlpy
!pip install soynlp
!apt -qq -y install fonts-nanum # 폰트 설치

In [ ]:
import pandas as pd 
import re
import pickle
from soynlp.utils import DoublespaceLineCorpus
from soynlp.noun import LRNounExtractor_v2
from konlpy.tag import Komoran, Okt
from collections import Counter
from wordcloud import WordCloud

okt = Okt()

file_path = '/content/drive/MyDrive/Colab Notebooks/3조_XXXX/04. 기타자료/data/'

# 데이터 불러오기

In [ ]:
talk_df = pd.read_csv(file_path + 'TalkSetTrain5.csv')
talk_df

,분류,문장,점수
0,혐오,부랴부랴 왔는데 아무도 안왔네. 시간개념들이 없네,10
1,혐오,맞아. 사람들이 진짜 개념이없다,10
2,혐오,인방 보는 남자는 거르는게 맞다,5
3,선정,특히 벗방보는 애들은 진짜 거세 시켜야함,10
4,혐오,댓글에 빠순이들 몰려와서 즈그 주인님 쉴드치는 꼴 좀 봐,10
...,...,...,...
222240,혐오,그것보다 어떻게 밥그릇 챙기면서 살 수 있을지 고민하는 게 훨씬 이득이지 않아?,8
222241,혐오,저 나이 쳐먹고는 아이돌 하겠다고 나대는거임? 세상이 잘 돌아간다,8
222242,혐오,저런 애들을 뽑아주기라도 하겠음? 그냥 실패의 맛 보러 나온거 아님? ㅋ,8
222243,비악플,저 여자는 오늘도 레깅스 입네,0


# Okt 명사 추출하기

In [ ]:
def Okt_ExtractNouns(series):
    noun_lst = []
    noun_set = set()
    for sent in series:
        nouns = okt.nouns(re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ0-9a-zA-Z\s]", "", sent))
        noun_lst.append(nouns)
        noun_set.update(nouns)
    
    return noun_lst, noun_set

In [ ]:
okt_nouns, okt_nouns_set = Okt_ExtractNouns(talk_df['문장'])

In [ ]:
print(f'Okt 형태소 분석기가 추출한 명사는 {len(okt_nouns_set)} 개 입니다.')

Okt 형태소 분석기가 추출한 명사는 29571 개 입니다.


In [ ]:
talk_df['NOUNS'] = okt_nouns
talk_df

,분류,문장,점수,NOUNS
0,혐오,부랴부랴 왔는데 아무도 안왔네. 시간개념들이 없네,10,"[부랴부랴, 아무, 시간, 개념]"
1,혐오,맞아. 사람들이 진짜 개념이없다,10,"[사람, 진짜, 개념]"
2,혐오,인방 보는 남자는 거르는게 맞다,5,"[인방, 남자, 거르는]"
3,선정,특히 벗방보는 애들은 진짜 거세 시켜야함,10,"[벗방, 애, 진짜, 거세]"
4,혐오,댓글에 빠순이들 몰려와서 즈그 주인님 쉴드치는 꼴 좀 봐,10,"[댓글, 빠순이, 즈그, 주인, 꼴, 좀]"
...,...,...,...,...
222240,혐오,그것보다 어떻게 밥그릇 챙기면서 살 수 있을지 고민하는 게 훨씬 이득이지 않아?,8,"[그것, 밥그릇, 살, 수, 고민, 게, 이득]"
222241,혐오,저 나이 쳐먹고는 아이돌 하겠다고 나대는거임? 세상이 잘 돌아간다,8,"[저, 나이, 아이돌, 임, 세상]"
222242,혐오,저런 애들을 뽑아주기라도 하겠음? 그냥 실패의 맛 보러 나온거 아님? ㅋ,8,"[저런, 애, 주기, 그냥, 실패, 맛]"
222243,비악플,저 여자는 오늘도 레깅스 입네,0,"[저, 여자, 오늘, 레깅스]"


# Soynlp 신조어 추출하기

In [ ]:
sentences = talk_df['문장'].apply(lambda x: re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ0-9a-zA-Z\s]", "", x)) 
sentences.to_csv(file_path + 'sentences.txt', index=False, header=False)

In [ ]:
sents = DoublespaceLineCorpus(file_path + 'sentences.txt', iter_sent=True)
noun_extractor = LRNounExtractor_v2(verbose=True, extract_compound=True)
nouns = noun_extractor.train_extract(sents)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 243384 from 223453 sents. mem=0.746 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=1393024, mem=1.152 Gb
[Noun Extractor] batch prediction was completed for 55598 words
[Noun Extractor] checked compounds. discovered 26389 compounds
[Noun Extractor] postprocessing detaching_features : 42117 -> 31927
[Noun Extractor] postprocessing ignore_features : 31927 -> 31708
[Noun Extractor] postprocessing ignore_NJ : 31708 -> 30760
[Noun Extractor] 30760 nouns (26389 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=1.266 Gb                    
[Noun Extractor] 59.91 % eojeols are covered


In [ ]:
def SoynlpExtractNouns(nouns, min_len=2, max_len=5, min_count=5, max_count=50, min_score=1):
    noun_lst = []
    for noun, score in nouns.items():
        if noun[-1] in ['님', '들']:
            noun = noun[:-1]

        if len(noun) >= min_len and len(noun) <= max_len:
            if score[0] >= min_count and score[0] <= max_count:
                if score[1] >= min_score and noun[0].isdigit()==False:
                    noun_lst.append([noun, score[0], score[1], len(noun)])
    
    noun_df = pd.DataFrame(noun_lst, columns=['noun', 'frequency', 'score', 'len'])
    noun_df = noun_df.sort_values(by=['frequency', 'len', 'noun', 'score'], ascending=False)
    noun_df.reset_index(inplace=True)
    
    return noun_df.drop(columns=['index'])


In [ ]:
soy_df = SoynlpExtractNouns(nouns, min_len=2, max_len=5, min_count=1, max_count=500, min_score=1)
soy_df

,noun,frequency,score,len
0,걔네,500.0,1.0,2
1,능력,499.0,1.0,2
2,미개,496.0,1.0,2
3,벌써,479.0,1.0,2
4,조금,452.0,1.0,2
...,...,...,...,...
21113,까더,1.0,1.0,2
21114,긍지,1.0,1.0,2
21115,금기,1.0,1.0,2
21116,것보,1.0,1.0,2


In [ ]:
soy_nouns_set = set(soy_df['noun'])

print(f'Okt 형태소 분석기가 추출한 명사는 {len(soy_nouns_set)} 개 입니다.')

Okt 형태소 분석기가 추출한 명사는 20757 개 입니다.


# 차집합 구하기

In [ ]:
diff_nouns_set = list(soy_nouns_set - okt_nouns_set)
diff_nouns_len = [len(noun) for noun in diff_nouns_set]

print(f'Okt 형태소 분석기가 추출한 명사는 {len(diff_nouns_set)} 개 입니다.')

Okt 형태소 분석기가 추출한 명사는 12591 개 입니다.


In [ ]:
# kiwi와 비교하기 위한 저장
with open(file_path + 'okt_soy_diff.pkl', 'wb') as f:
    pickle.dump(diff_nouns_set, f)

In [ ]:
print(diff_nouns_set)

['술주정', '여론통제', '영화봤는데', '똥꼬치마', '싸패같고', '깨끗한척', '할아버지뻘', '아닌가봐', '야시시', '우유급식', '뒷담', '목줄달고', '피해니까', '집안꼴', '납작가슴', '공익이', '늘어놓지', '공사장', '싹수보면', '꼴릿한게', '살림살', '대통령탄핵', '합법화하고', '대화주제', '친해보', '불친절하고', '이해가도', '냥줍', '말도안', '과잉의식', '다음수업', '뒷방틀딱', '노인회장', '생긴말', '직업윤리', '개인카드', '틀지', '개인계좌', '한심이', '임대충', '적이', '여군시험', '쓸때', '공손', '밀어버릴까', '노력한', '친일끼리', '검색하고', '위험한건데', '귀엽기만', '소홀히', '폐지청원', '비행있으면', '말만하고', '명의도용', '냄새나긴', '국밥챔프', 'K구걸', '무임승차', '순대국', '남말', '띄어', '군바리ㅅㄲ', '전재산', '듣지', '한둘', '양심고백', '성매매여성', '뿌린데', '참견질', '이딴소리', '쳐넣어야', '인공지능', '많은데', '빡빡하고', '생고생', '극심', '장사예정', '헛소리ㅉㅉ', '먹었어야', '댓글창', '젖통까고', '얼어죽게', '태워줘야', '한남페이', '와입', '내눈', '호감형', '어깨피고', '시한부', '제작자가', '불쌍한건', '바람구멍', '두다리', '탱글탱글', 'ㅇㅇ근데', '까고', '안내고', '특수반', 'ㅡㅡ', '그모양인거', '백댄서', '기억삭제', '감수하는것', '소녀상', '개인노동', '도망쳐야', '저애', '군대가고', '쌍방폭행', '마음고생', '선수급', '모기때문', '껌딱지', '장애인복지', '대구무시', '양키님', '인권주장', '세뇌교육', '싶은건', '코박죽하고', '아이들끼리', '특별고용', '훤칠', '순종적', '도배치장', '머리안', '패죽여야', '지옥길', '돌싱관련', '팍팍', '변기통',

In [ ]:
# 검수를 위한 데이터 저장
diff_nouns_df = pd.concat([pd.Series(diff_nouns_set, name='noun'), pd.Series(diff_nouns_len, name='len')], axis=1)
diff_nouns_df.sort_values(by=['len', 'noun'], ascending=True, inplace=True)
diff_nouns_df.drop(columns=['len'], inplace=True)
diff_nouns_df.to_csv(file_path + 'okt_soy_nouns.txt', index=False, header=False)

# 검수 후 신조어 관리

In [ ]:
edit_nouns = set()
with open(file_path + 'okt_soy_nouns.txt', 'r') as f:
    edit_nouns.update(f.read().split('\n'))

print(f'등록할 신조어 명사는 {len(edit_nouns)}개 입니다.')

등록학 신조어 명사는 4275개 입니다.


In [ ]:
edit_etcs = set()
with open(file_path + 'okt_soy_etc.txt', 'r') as f:
    edit_etcs.update(f.read().split('\n'))

print(f'등록할 신조어 명사는 {len(edit_etcs)}개 입니다.')

등록할 신조어 명사는 410개 입니다.
